# Demux samples

refer to [`seperate_samples_helper.py`](https://github.com/cvraut/Gecko_screen2/blob/668b2726668f526915199cc473adf0d3bc12b701/code/p15_analyze_9536_data/seperate_samples_helper.py) for the core logic.

Notable changes to the demux logic. Because these are paired-end reads we actually get 2 mappings per read. This notebook will apply the simple technique to the forward read pairs first and then try to use the paired nature of the reads to be double sure of the alignment.

# Old unpaired logic

In [4]:
import pandas as pd

s = "/home/craut/wkspce/CRISPRai_NGS/data_out/15107-PP/alignment/15107-PP-1_S1_001_pe.sam" # sam file location
c = set(["6S30M115S"]) # cigar strings to screen for (the good reads)
o = "/home/craut/wkspce/CRISPRai_NGS/data_out/15107-PP/alignment/unpaired_demux_R1_only/15107_PP_sample_demux" # header for the output files
debug=True

In [5]:
if debug:
    print(f"samfile: {s}")
    print(f"cigars: {c}")
    print(f"output_header: {o}")

header_lines = 0
with open(s) as f:
    for i,line in enumerate(f):
        if line.startswith("@"):
            header_lines += 1
        else:
            break
sam_col_names = ["QNAME","FLAG","RNAME","POS","MAPQ","CIGAR","RNEXT","PNEXT","TLEN","SEQ","QUAL","AS","XS","XN","XM","XO","XG","NM","MD","YT"]
if debug:
    print(f"header_lines: {header_lines}")
sam_results = pd.read_table(s,header=None,skiprows=header_lines, names=sam_col_names, index_col=False)
if debug:
    print(sam_results.shape)
unmapped = sam_results[sam_results["RNAME"] == "*"]
# write the unmapped reads to a file
with open(f"{o}_unmapped.fastq","w+") as of:
    for qname, seq, qual in zip(unmapped["QNAME"],unmapped["SEQ"],unmapped["QUAL"]):
        of.write(f"@{qname}\n{seq}\n+\n{qual}\n")
del unmapped
# filter out the reads that don't have the correct cigar string
sam_results = sam_results[sam_results["CIGAR"].isin(c)]
# sort the reads by the RNAME
sam_results.sort_values(by=["RNAME"],inplace=True)
barcode,of = None,None
for qname, rname, seq, qual in zip(sam_results["QNAME"],sam_results["RNAME"],sam_results["SEQ"],sam_results["QUAL"]):
    if barcode != rname:
        if barcode:
            of.close()
        barcode = rname
        of = open(f"{o}_{rname}.fastq","w+")
    of.write(f"@{qname}\n{seq}\n+\n{qual}\n")

samfile: /home/craut/wkspce/CRISPRai_NGS/data_out/15107-PP/alignment/15107-PP-1_S1_001_pe.sam
cigars: {'6S30M115S'}
output_header: /home/craut/wkspce/CRISPRai_NGS/data_out/15107-PP/alignment/unpaired_demux_R1_only/15107_PP_sample_demux
header_lines: 6


/tmp/ipykernel_89631/842986436.py:16: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  sam_results = pd.read_table(s,header=None,skiprows=header_lines, names=sam_col_names, index_col=False)


(14398310, 20)


In [6]:
sam_results 

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,AS,XS,XN,XM,XO,XG,NM,MD,YT
0,M06643:669:000000000-GVW9B:1:1101:15772:1330,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-228,AGCATAACAGTGTCTTGTGGAAAGGACGAAACACCGCACATCGCCC...,BBBBBFFFFFFFGGGGGBFEFGHGGFGGGGGGGHGGGGGGHHGEEG...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:57
11401180,M06643:669:000000000-GVW9B:1:2103:7912:10256,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-222,ACGGTCACAGTGTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,ABBBAABFFFFFGGGGGGGGGGHHHHHGGGGGHHGGGGGGHGGGGG...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:57
3657608,M06643:669:000000000-GVW9B:1:1102:13996:27609,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-222,TTACGCACAGTGTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,AAABABCCCFFFGGGGGGGGGGHHHHHGGGGGHHGGAA221A0AEF...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:60
3657600,M06643:669:000000000-GVW9B:1:1102:16945:27609,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-228,CCTTAAACAGTGTCTTGTGGAAAGGACGAAACACCGCACATCGCCC...,BBCCCFFFFFFFGGGGGGGGGGHHHHHGGGGGHHGGGGGGHHGGGG...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:60
3657596,M06643:669:000000000-GVW9B:1:1102:13874:27628,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-224,TTGCTTACAGTGTCTTGTGGAAAGGACGAAACACCGAGCGCACATC...,AABAAFFFFF5@FGGGGFGGGGHHHHHGGGGGHHGGGGGGGEGGGE...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6099140,M06643:669:000000000-GVW9B:1:1104:27770:11037,99,barcode_TGACCA,1,2,6S30M115S,=,1,-224,CTCGCATGACCATCTTGTGGAAAGGACGAAACACCGCACGCACATC...,DDDDDCCCFFFFGGGGGGGGGGHHHHHGGGGGHHGGGGGGGGGGHH...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:57
2385858,M06643:669:000000000-GVW9B:1:1102:8476:10563,73,barcode_TGACCA,1,2,6S30M115S,=,1,0,TCATATTGACCATCTTGTGGAAAGGACGAAACACCGCTAGTGAGGC...,CCDCDFFFFFFFGGGGGGGGGGHHHHHGGGGEHHGGGGGGHHGHHG...,AS:i:60,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YT:Z:UP,NaN
2385848,M06643:669:000000000-GVW9B:1:1102:7634:10563,99,barcode_TGACCA,1,2,6S30M115S,=,1,-224,AGTTTATGACCATCTTGTGGAAAGGACGAAACACCGCACGCACATC...,BCACCFFFFFFFGGGGGGGGGGHHHHHGGGGGHHGGGGGGGGGGHH...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:60
6099040,M06643:669:000000000-GVW9B:1:1104:23264:11035,73,barcode_TGACCA,1,2,6S30M115S,=,1,0,GATAGTTGACCATCTTGTGGAAAGGACGAAACACCGGGTGCGACGT...,CBCCCFFFFFFFGGGGGGGGGGHHHGHGGGGGHHGGFGEFGGEGGG...,AS:i:60,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YT:Z:UP,NaN


# Explore relationship of paired reads

Refer to https://en.wikipedia.org/wiki/SAM_(file_format) for what the bitset in the FLAG column corresponds to and how to interpret the sam results.

In [7]:
sam_results_original = pd.read_table(s,header=None,skiprows=header_lines, names=sam_col_names, index_col=False)

/tmp/ipykernel_89631/3993947524.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  sam_results_original = pd.read_table(s,header=None,skiprows=header_lines, names=sam_col_names, index_col=False)


In [8]:
sam_results_original

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,AS,XS,XN,XM,XO,XG,NM,MD,YT
0,M06643:669:000000000-GVW9B:1:1101:15772:1330,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-228,AGCATAACAGTGTCTTGTGGAAAGGACGAAACACCGCACATCGCCC...,BBBBBFFFFFFFGGGGGBFEFGHGGFGGGGGGGHGGGGGGHHGEEG...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:57
1,M06643:669:000000000-GVW9B:1:1101:15772:1330,147,barcode_ACAGTG,1,2,83S30M38S,=,1,228,TTTTCTCTTTTTTTTTTTTAATGATACGGCGCCCACCGAGATCTAC...,/;://..-CC<@@>?11?1@?100<B<//>//EB?AFB21F2B>BB...,AS:i:57,XS:i:45,XN:i:0,XM:i:1,XO:i:0,XG:i:0,NM:i:1,MD:Z:20C9,YS:i:60
2,M06643:669:000000000-GVW9B:1:1101:15740:1330,99,barcode_TGACCA,1,2,6S30M115S,=,1,-224,AGATCCTGACCATCTTGTGGAAAGGACGAAACACCGCACGCACATC...,CCBCCFFFFFFFGGGGGGGGGGHHGHHGGGGGHHGGGGGGGGGGHH...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:57
3,M06643:669:000000000-GVW9B:1:1101:15740:1330,147,barcode_TGACCA,1,2,79S30M42S,=,1,224,TTTTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTACACTC...,9;:--C@CCGFFFGFHGDHFFCGGCCF??EAGHHHHHHGHGB>GGG...,AS:i:57,XS:i:45,XN:i:0,XM:i:1,XO:i:0,XG:i:0,NM:i:1,MD:Z:24A5,YS:i:60
4,M06643:669:000000000-GVW9B:1:1101:15889:1332,99,barcode_ACAGTG,1,2,6S30M115S,=,1,-217,TGCTGCACAGTGTCTTGTGGAAAGGACGAAACACCGCCGCTAGAGC...,CCCCCFFFFFFFGGGGGGGGGGHHHGHGGGGGHHGGGGEEEGGHHH...,AS:i:60,XS:i:48,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14398305,M06643:669:000000000-GVW9B:1:2104:14623:29062,147,barcode_GCCAAT,1,2,77S30M44S,=,1,222,TTTTTTTTTTTTTAACGATACGGCGACCACCGAGATCTACACTCTT...,------<CBGF..F>.<<<><</A?/ACEFFFF@2BBBB1FFHGFF...,AS:i:54,XS:i:44,XN:i:0,XM:i:2,XO:i:0,XG:i:0,NM:i:2,MD:Z:24A1A3,YS:i:60
14398306,M06643:669:000000000-GVW9B:1:2104:17599:29063,99,barcode_GCCAAT,1,2,6S30M115S,=,1,-222,CTCATCGCCAATTCTTGTGGAAAGGACGAAACACCGAGCGCGGACA...,AAAAAB11A111FGGGGGGGGGHHHHHGGGGGHHGG/////A/A>/...,AS:i:60,XS:i:50,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:54
14398307,M06643:669:000000000-GVW9B:1:2104:17599:29063,147,barcode_GCCAAT,1,2,77S30M44S,=,1,222,CTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTACACTCTT...,.-;-----G0=00<>10CC<<.C<?</??CF>2F2>D>?1??0FGB...,AS:i:54,XS:i:42,XN:i:0,XM:i:2,XO:i:0,XG:i:0,NM:i:2,MD:Z:24A1A3,YS:i:60
14398308,M06643:669:000000000-GVW9B:1:2104:16052:29063,99,barcode_GCCAAT,1,2,6S30M115S,=,1,-222,TACCGCGCCAATTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,BBB@ABBBDABFGFGGBGGGGGHHHFHGGGGGHHGGGGGGCGGGGG...,AS:i:60,XS:i:50,XN:i:0,XM:i:0,XO:i:0,XG:i:0,NM:i:0,MD:Z:30,YS:i:54


In [13]:
from tqdm.autonotebook import tqdm
from collections import Counter, defaultdict

In [18]:
read_names = set(sam_results_original["QNAME"])

read_names_2_data = defaultdict(list)
for qname, flag, rname, cigar, seq in tqdm(zip(sam_results_original["QNAME"], sam_results_original["FLAG"], sam_results_original["RNAME"], sam_results_original["CIGAR"], sam_results_original["SEQ"]),total=sam_results_original.shape[0]):
    read_names_2_data[qname].append( (flag, rname, cigar, seq) )

read_info = []

for read_name in tqdm(read_names):
    
    datum = {}
    datum["QNAME"] = read_name
    datum["num_alignments"] = len(read_names_2_data[read_name])
    datum["forward_sequence"] = None
    datum["reverse_sequence"] = None
    datum["forward_mapping"] = None
    datum["reverse_mapping"] = None
    datum["forward_cigar"] = None
    datum["reverse_cigar"] = None
    datum["notes"] = ""
    for flag, rname, cigar, seq in read_names_2_data[read_name]:
        if flag & 128: # reverse read
            if datum["reverse_sequence"] is not None:
                datum["notes"] += "multiple_reverse_alignments,"
            else:
                datum["reverse_sequence"] = seq
                datum["reverse_mapping"] = rname
                datum["reverse_cigar"] = cigar
                if flag & 4:
                    datum["notes"] += "reverse_unmapped_read,"
                if not flag & 2 and "not_properly_paired," not in datum["notes"]:
                    datum["notes"] += "not_properly_paired,"
        elif flag & 64: # forward read
            if datum["forward_sequence"] is not None:
                datum["notes"] += "multiple_forward_alignments,"
            else:
                datum["forward_sequence"] = seq
                datum["forward_mapping"] = rname
                datum["forward_cigar"] = cigar
                if flag & 4:
                    datum["notes"] += "forward_unmapped_read,"
                if not flag & 2 and "not_properly_paired," not in datum["notes"]:
                    datum["notes"] += "not_properly_paired,"
    read_info.append(datum)
read_info_df = pd.DataFrame(read_info)
print(read_info_df.shape)

  0%|          | 0/14398310 [00:00<?, ?it/s]

  0%|          | 0/7199155 [00:00<?, ?it/s]

(7199155, 9)


In [19]:
read_info_df

,QNAME,num_alignments,forward_sequence,reverse_sequence,forward_mapping,reverse_mapping,forward_cigar,reverse_cigar,notes
0,M06643:669:000000000-GVW9B:1:1103:15655:21416,2,TGTGTTGCCAATTCTTGTGGAAAGGACGAAACACCGGGCAGTGAGT...,TNTANTATNCTTTCCNNTGNANTGNACCCCCCAATCCCCCCTTTTC...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,*,"not_properly_paired,reverse_unmapped_read,"
1,M06643:669:000000000-GVW9B:1:2102:11629:19558,2,TTCAATTGACCATCTTGTGGAAAGGACGAAACACCGCACGCACATC...,TTTTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTACACTC...,barcode_TGACCA,barcode_TGACCA,6S30M115S,79S30M42S,
2,M06643:669:000000000-GVW9B:1:2102:15431:23346,2,CCAGGTGCCAATTCTTGTGGAAAGGACGAAACACCGCCCCAGCGGC...,TNNACTATNCTTTCCCNTGNANNGNACTCCCCAATGCCCCCTTTTC...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,*,"not_properly_paired,reverse_unmapped_read,"
3,M06643:669:000000000-GVW9B:1:1101:18905:10121,2,TAGTGGGCCAATTCTTGTGGAAAGGACGAAACACCGCAGGACATCA...,TNTANTATTCTTTCCNCTGCACTGTACCCCCCAATCCCCCCTTTTC...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,*,"not_properly_paired,reverse_unmapped_read,"
4,M06643:669:000000000-GVW9B:1:2101:23487:11854,2,ATCGGAGCCAATTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,TTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTACACTCTT...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,77S30M44S,
...,...,...,...,...,...,...,...,...,...
7199150,M06643:669:000000000-GVW9B:1:1103:14309:17295,2,TTATCTGCCAATTCTTGTGGAAAGGACGAAACACCGCCCCGGGCCT...,TNTANTATNCTTTCCNNTGNANTGNACCCCCCAATTCCCCCTTTTC...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,*,"not_properly_paired,reverse_unmapped_read,"
7199151,M06643:669:000000000-GVW9B:1:2101:12005:2633,2,TATGAAGCCAATTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,TTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTACACTCTT...,barcode_GCCAAT,barcode_GCCAAT,6S30M115S,77S30M44S,
7199152,M06643:669:000000000-GVW9B:1:2102:23852:21294,2,GTGTCTACAGTGTCTTGTGGAAAGGACGAAACACCGAGAGCGCACA...,TTTTTTTTTTTTTAATGATACGGCGACCCCCGAGATCTACACTCTT...,barcode_ACAGTG,barcode_ACAGTG,6S30M115S,77S30M44S,
7199153,M06643:669:000000000-GVW9B:1:1102:28122:14198,2,TTCATTACAGTGTCTTGTGGAAAGGACGAAACACCGCACATCGCCC...,CGTTTACTTTTTTTTTTTTAATGATACGGCGACCACCGAGATCTAC...,barcode_ACAGTG,barcode_ACAGTG,6S30M115S,83S30M38S,


In [25]:
# count the number of properly paired and matched reads
properly_paired_and_matched = read_info_df[(read_info_df["notes"] == "") & pd.Series([cs.count("M") == 1 and "30M" in cs for cs in read_info_df["forward_cigar"]])]
num_properly_paired_and_matched = properly_paired_and_matched.shape[0]
total_num_reads = len(read_names)
num_reads_unpaired = sam_results.shape[0]
print(f"Total number of reads: {total_num_reads}")
print(f"Number of unpaired reads with CIGAR={c}: {num_reads_unpaired} ({num_reads_unpaired/total_num_reads:.2%})")
print(f"Number of properly paired and matched reads: {num_properly_paired_and_matched} ({num_properly_paired_and_matched/total_num_reads:.2%})")

Total number of reads: 7199155
Number of unpaired reads with CIGAR={'6S30M115S'}: 6112455 (84.91%)
Number of properly paired and matched reads: 5316229 (73.85%)


Based on these results, we should only use the forward strand reads. We get almost 15% more reads if we abandon the paired sequence nature of the reads.

In [30]:
my_cnts = Counter(properly_paired_and_matched["forward_cigar"])
my_cnts

Counter({'6S30M115S': 5280129,
         '7S30M114S': 6545,
         '5S30M116S': 27889,
         '4S30M117S': 911,
         '28S30M93S': 6,
         '2S30M119S': 105,
         '3S30M118S': 181,
         '46S30M75S': 4,
         '52S30M69S': 5,
         '34S30M87S': 10,
         '42S30M79S': 9,
         '58S30M63S': 4,
         '37S30M84S': 3,
         '56S30M65S': 8,
         '1S30M120S': 25,
         '25S30M96S': 2,
         '13S30M108S': 6,
         '8S30M113S': 53,
         '16S30M105S': 13,
         '63S30M58S': 3,
         '50S30M71S': 9,
         '71S30M50S': 1,
         '12S30M109S': 10,
         '19S30M102S': 9,
         '30M121S': 29,
         '45S30M76S': 4,
         '10S30M111S': 17,
         '9S30M112S': 11,
         '90S30M31S': 1,
         '44S30M77S': 7,
         '31S30M90S': 6,
         '20S30M101S': 7,
         '53S30M68S': 5,
         '11S30M110S': 6,
         '60S30M61S': 4,
         '14S30M107S': 21,
         '26S30M95S': 4,
         '18S30M103S': 8,
         '48S30

In [31]:
print(f"When considering fuzzy or shifted matches we only increase the number of reads by {num_properly_paired_and_matched/my_cnts['6S30M115S']-1:.2%} ({num_properly_paired_and_matched-my_cnts['6S30M115S']} reads)")

When considering fuzzy or shifted matches we only increase the number of reads by 0.68% (36100 reads)


Considered shifted matches does not significantly increase the number of reads, so using the CIGAR of `6S30M115S` should suffice for downstream work.